### Analysing Used Car Data from Ebay

We are going to work on a dataset of used cars listed  from Ebay Kleinanzeigen, the German based version of Ebay.  

The original data was originally scraped from [Orges Leka database](https://www.kaggle.com/orgesleka/used-cars-database), The version of the dataset we will be working with is a sample of the first 50,000 data points to run the code faster. That will not affect our analysis.

In [ ]:
import pandas as pd
import numpy as np
autos = pd.read_csv(r"../input/autos.csv", encoding="Windows-1252", nrows=50000)

In [ ]:
autos.isnull().sum()

The dataset contains 20 columns, most of which are strings.
Some columns have null values, but none have more than ~20% null values.
The column names use camelcase instead of Python's preferred snakecase, which means we can't just replace spaces with underscores.

In [ ]:
cols = ["date_crawled", "name", "seller", "offer_type", "dollar_price", "ab_test",
        "vehicle_type", "registration_year", "gearbox", "power_ps", "model",
       "kilometer", "registration_month", "fuel_type", "brand", "unrepaired_damage",
        "ad_created", "pictures_number", "postal_code",  "last_seen_online"]
autos.columns = cols

In [ ]:
autos.head()

In [ ]:
print(autos["pictures_number"].sum())
print(autos["ab_test"].unique())
print(autos["offer_type"].unique())
print(autos['seller'].unique())
autos.drop(["pictures_number", "ab_test", "offer_type", "seller"], axis=1, inplace=True)

Based on the information we got from the data within each column, we decide to delete the following ones:
 - pictures_number because all  values are equal to zero
 - ab_test takes two different values: 'test' and 'control' that are not relevant for our analysis
 - offer_type takes two different values 'Angebot' and 'Gesuch' that are not relevant for our analysis
 - seller takes only one value

In [ ]:
autos.info()

The df.infos() method gives us information about the number of observations, each column data type and the memory used by the df. We can have more details about numerical values with the df.describe() method.

In [ ]:
autos.describe()

Based on min and max values of numerical data above, we notice that we can reduce the memory usage by storing the data in more optimal types. For example, prices, kilometers and registration years are all positive values, we can use unsigned integers to store them instead of int64 and reduce the memory allocation.

In [ ]:
autos.dollar_price = autos.dollar_price.astype(np.uint32)
autos.kilometer = autos.kilometer.astype(np.uint32)
autos.power_ps = autos.power_ps.astype(np.uint16)
autos.registration_month = autos.registration_month.astype(np.uint8)
autos.registration_year = autos.registration_year.astype(np.uint16)
autos.postal_code = autos.postal_code.astype(np.uint32)

In [ ]:
autos.info()

We then reduced the memory usage by 1.5 MB out of 6.1 MB, i.e. 25%, just by optimizing the types of numerical data for 50,000 rows. 

In [ ]:
print(autos.dollar_price.value_counts().sort_index().head())
print(autos.dollar_price.unique().shape)
c_dollar_prices = autos.dollar_price.copy()
s_dollar_prices = c_dollar_prices.sort_values(ascending=False)
s_dollar_prices.index = autos.index
print(s_dollar_prices.head())

There is exactly 2381 different prices from $0 to $12345678. We can easily deduce that some of these values are wrong and will bias our analysis. Let's have a deeper check to decide which range of prices we will keep.

In [ ]:
v_dollar_prices = autos[autos["dollar_price"] > 100000].copy()
v_dollar_prices.sort_values(by=['dollar_price'], ascending=False, inplace=True)
v_dollar_prices[['name', 'kilometer', 'dollar_price']].head(20)

### What happened this time?

The first part of the above operation selects the price column. When pandas selects a single column from a DataFrame, pandas creates a view and not a copy. A view just means that no new object has been created. df['price'] references the price column in the original DataFrame.

This is analogous to the list example where we assigned an entire list to a new variable. No new object is created, just a new reference to the one already in existence.

Since no new data has been created, the assignment will modify the original DataFrame.

### Why is a warning triggered when our operation completed successfully?

Pandas does not know if you want to modify the original DataFrame or just the first subset selection.

For instance, you could have selected the price column as a Series to do further analysis with it without affecting the original DataFrame.

To get an independent copy, we called the copy method. 

In [ ]:
v_dollar_prices._is_copy is None

v_dollar_prices does not refer the original dataframe, we can then make new assignments without modifying the original dataframe and without getting the SettingWithCopy Warning.

In [ ]:
autos.dollar_price.value_counts().sort_index().head(20)

There are a number of listings with prices below \$40, including about 1,500 at \$0. There are also a small number of listings with very high values, including 9 at around or over $1 million.

Given that eBay is an auction site, there could legitimately be items where the opening bid is \$1. The opening bid does not however represent the real value of a car, we then decide to fix the minimal price for a car to $500 because we need prices to be real if we want to make some comparisons between brands for example. We remove anything above \$245,000, since it seems that prices increase steadily to that number and then double to next number.

In [ ]:
print((~autos["dollar_price"].between(500,245000)).sum() / autos.shape[0])
autos = autos[autos["dollar_price"].between(500,245000)]
v_dollar_prices = autos[autos["dollar_price"] >= 50000].copy()
v_dollar_prices.sort_values(by=['dollar_price'], ascending=False, inplace=True)
v_dollar_prices.head(20)

~10% of the data have been removed which represents a important number of outliers.

In [ ]:
autos.loc[(autos["vehicle_type"] == "kombi") & (autos["dollar_price"] > 50000), "dollar_price"]

In [ ]:
autos.index

In [ ]:
autos.index = range(45081)

As we removed data from the original data frame, it's cleaner to reset the index as we did in the cell above.

#### View of the various vehicle and fuel types

In [ ]:
print(autos['vehicle_type'].unique(), autos['fuel_type'].unique())

#### We use a dictionary and the map function to convert vehicle and fuel types from German to English

In [ ]:
mapping_vehicle_type = {
    'coupe': 'coupé',
    'kleinwagen': 'small car',
    'kombi': 'station wagon',
    'cabrio': 'convertible',
    'andere': 'other',
    'hybrid':'hybrid',
    'limousine':'limousine',
    'bus':'bus'
                        }

mapping_fuel_type = {
    'benzin': 'gasoline',
    'diesel': 'diesel',
    'lpg': 'liquefied petroleum gas',
    'andere': 'other',
    'cng': 'compressed natural gas',
    'elekto':'electric'
                    }

autos['vehicle_type'] = autos['vehicle_type'].map(mapping_vehicle_type)
autos['fuel_type'] = autos['fuel_type'].map(mapping_fuel_type)

#### Exploring the dates columns

There are 5 columns that should represent date values. Some of these columns were created by the crawler, some came from the website itself. We can differentiate by referring to the data dictionary:

- `date_crawled`: added by the crawler
- `last_seen`: added by the crawler
- `ad_created`: from the website
- `registration_month`: from the website
- `registration_year`: from the website

In [ ]:
autos[['date_crawled','ad_created','last_seen_online']].head()

You'll notice that the first 10 characters represent the day (e.g. 2016-03-12). To understand the date range, we can extract just the date values, use Series.value_counts() to generate a distribution, and then sort by the index.

To select the first 10 characters in each column, we can use Series.str[:10]

In [ ]:
date_col = ['date_crawled', 'ad_created', 'last_seen_online']
for col in date_col:
    autos[col] = autos[col].str[:10]

In [ ]:
autos['date_crawled'].value_counts(normalize=True).sort_index(ascending=True)

The cars were crawled each day between March 2016, 5th and March 2016, 7th. The repartition of the data collection is uniform, each day ~3% of the data were collected.

In [ ]:
a = autos['ad_created'].value_counts(normalize=True).sort_index(ascending=False)
b = a[a > 0.02]
b

The ads crawled were created between March 2015, 20th and April 2016, 7th. We notice that 93% of them were created between March 2016, 5th and April 2016, 7th. It's in line with the dates the website was crawled. We can infer that an ad has a lifetime of approximately one month. We could for example neglect the ads created before March 2016 depending on our analysis.

In [ ]:
autos['last_seen_online'].value_counts(normalize=True).sort_index(ascending=False)

~50% of the cars were last seen online during the three last days the website was crawled. We can infer that the crawler needs to put a value in the last_seen_online column even if the ad is still active. If we have to analyse the car sales, we will then exclude these three days from the analysis.

Let's check the consistency of the data and the reliability of the crawler by comparing the date the ad was created and the last time it was seen online:

In [ ]:
(autos['ad_created'] <= autos['last_seen_online']).all()

All the ads were created before they were last seen online, then the crawling worked properly.

In [ ]:
a = (autos['registration_year'] > 2016)
b = a[a == True]
b.count()

Some of the data in the registration_year column are wrong, a registration superior to April 2016 is not possible based on the date of the latest crawling. Given  the important number of cars between 2017 and 2019, we could change these values to 2016 for example.  We can consider that a registration before 1945 is also an error. Even if some ads might be right, there is a high probabilty to find outliers.

In [ ]:
cars_by_year = autos['registration_year'].value_counts().sort_index(ascending=False)
autos.loc[autos["registration_year"].between(2017,2018), "registration_year"] = 2016
print(cars_by_year.head(12))
print(cars_by_year.tail(12))

In [ ]:
(~autos["registration_year"].between(1945,2016)).sum() / autos.shape[0]

The percentage of values that we are going to remove from the dataset is marginal (less than 0.1%)

In [ ]:
# Many ways to select rows in a dataframe that fall within a value range for a column.
# Using `Series.between()` is one way.
autos = autos[autos["registration_year"].between(1945,2016)]
autos["registration_year"].value_counts(normalize=True).head(10)

In [ ]:
p_brands = autos['brand'].value_counts(normalize=True)
p_brands

To carry out the analysis of the car price within each brand, we are going to consider only brands that represent at least 1% of the whole dataset, there are 16 brands. We need enough data to perform this kind of analysis to avoid mistaking due to an inconsistency of the data. A wrong price has more impact if the number of total prices is low.

In [ ]:
brand_mean_price = {}
brands = p_brands[p_brands > 0.01].index
for b in brands:
    mean_price = int(autos.loc[autos.brand == b, "dollar_price"].mean())
    mean_mileage = int(autos.loc[autos.brand == b, "kilometer"].mean())
    mean_year = int(autos.loc[autos.brand == b, "registration_year"].mean())
    brand_mean_price[b] = [mean_price, mean_mileage, mean_year]
df = pd.DataFrame.from_dict(brand_mean_price, orient='index', columns=['mean_price', 'mean_mileage', 'mean_year'])
df.head()

The mean prices are in line with our knowledge on the brand real prices. The three more expensive brands are respectively Audi, Bmw and Mercedes because of their high standards. More affordable brands like Renault, Fiat, Opel and Peugeot have a cheaper mean price. We could also add the mean year of registration per brand to improve the analysis. 